# Aggregating all player data for in-depth analyses and comparisons

In [4]:
# libraries
import pandas as pd
import numpy as np
import time
import re

#### Getting the standard stats for all players playing for clubs from the Top5 leagues in 2018/19

In [5]:
# creating list of urls for each top5 league club from the 18-19 season
url_list = open('urls/matchday_reports.txt').read().splitlines()
print(url_list[-1])
print(len(url_list))

https://fbref.com/en/matches/96de6624/Tottenham-Hotspur-Liverpool-June-1-2019-Champions-League
51


In [6]:
tables = pd.read_html(url_list[-1], attrs={"id": "shots_822bd0ba"})
df = tables[0]
df

Unnamed: 0_level_0      Unnamed: 1_level_0 Unnamed: 2_level_0  \
               Minute                  Player              Squad   
0                 2.0     Mohamed Salah (pen)          Liverpool   
1                17.0  Trent Alexander-Arnold          Liverpool   
2                21.0           Mohamed Salah          Liverpool   
3                22.0           Mohamed Salah          Liverpool   
4                38.0        Andrew Robertson          Liverpool   
5                39.0           Mohamed Salah          Liverpool   
6                42.0        Jordan Henderson          Liverpool   
7                42.0           Mohamed Salah          Liverpool   
8                 NaN                     NaN                NaN   
9                53.0                 Fabinho          Liverpool   
10               54.0           Mohamed Salah          Liverpool   
11               69.0            James Milner          Liverpool   
12               83.0        Jordan Henderson          Liverpool   
13               87.0         Virgil van Dijk          Liverpool   
14               87.0            Divock Origi          Liverpool   

   Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0  \
                   xG               PSxG            Outcome   
0                0.79               0.91               Goal   
1                0.02                NaN         Off Target   
2                0.03                NaN            Blocked   
3                0.04                NaN         Off Target   
4                0.03               0.06              Saved   
5                0.03                NaN         Off Target   
6                0.03                NaN            Blocked   
7                0.02                NaN            Blocked   
8                 NaN                NaN                NaN   
9                0.03                NaN            Blocked   
10               0.11                NaN            Blocked   
11               0.05                NaN         Off Target   
12               0.03                NaN         Off Target   
13               0.05                NaN            Blocked   
14               0.04               0.58               Goal   

   Unnamed: 6_level_0 Unnamed: 7_level_0 Unnamed: 8_level_0  \
             Distance          Body Part              Notes   
0                13.0          Left Foot                NaN   
1                33.0         Right Foot                NaN   
2                27.0          Left Foot                NaN   
3                16.0          Left Foot                NaN   
4                24.0          Left Foot                NaN   
5                23.0         Right Foot             Volley   
6                32.0         Right Foot                NaN   
7                28.0          Left Foot             Volley   
8                 NaN                NaN                NaN   
9                30.0         Right Foot                NaN   
10               12.0          Left Foot                NaN   
11               17.0          Left Foot                NaN   
12               28.0         Right Foot                NaN   
13               15.0         Right Foot             Volley   
14               15.0          Left Foot                NaN   

                     SCA 1                          SCA 2               
                    Player        Event            Player        Event  
0                      NaN       Fouled  Jordan Henderson  Pass (Live)  
1         Jordan Henderson  Pass (Live)           Fabinho  Pass (Live)  
2          Roberto Firmino  Pass (Live)     Mohamed Salah  Pass (Live)  
3   Trent Alexander-Arnold  Pass (Dead)     Mohamed Salah         Shot  
4               Joël Matip  Pass (Live)           Alisson  Pass (Live)  
5         Andrew Robertson         Shot               NaN          NaN  
6                      NaN          NaN               NaN          NaN  
7         Jordan Henderson  Pass (Live)   Rob

In [15]:
from urllib.parse import urlparse
# Initialize an empty list to store dataframes
dfs = []

# Set the delay between requests (in seconds)
delay_between_requests = 5  # Adjust this value as needed

for url in url_list:
    try:
        # Read HTML tables from the URL
        tables = pd.read_html(url, attrs={"id": "shots_all"}) # only Liverpool: "id": "shots_822bd0ba"
        df = tables[0]
        
        # Extract the date from the URL using regex
        date_match = re.search(r'(\w+-\d{1,2}-\d{4})', url)
        if date_match:
            date = pd.to_datetime(date_match.group(1), format="%B-%d-%Y").strftime("%Y-%m-%d")
        else:
            raise ValueError("Date not found in URL")
        
        # Add the Date column to the DataFrame
        df['Date'] = date
        
        # append df to the list
        dfs.append(df)
        
        # Introduce a delay before making the next request
        time.sleep(delay_between_requests)
    except Exception as e:
        print(f"Error reading data from {url}: {str(e)}")

# Concatenate all dataframes into one
total_df = pd.concat(dfs, ignore_index=True)

In [16]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260 entries, 0 to 1259
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   (Unnamed: 0_level_0, Minute)     1209 non-null   object 
 1   (Unnamed: 1_level_0, Player)     1209 non-null   object 
 2   (Unnamed: 2_level_0, Squad)      1209 non-null   object 
 3   (Unnamed: 3_level_0, xG)         1208 non-null   float64
 4   (Unnamed: 4_level_0, PSxG)       426 non-null    float64
 5   (Unnamed: 5_level_0, Outcome)    1209 non-null   object 
 6   (Unnamed: 6_level_0, Distance)   1209 non-null   float64
 7   (Unnamed: 7_level_0, Body Part)  1209 non-null   object 
 8   (Unnamed: 8_level_0, Notes)      183 non-null    object 
 9   (SCA 1, Player)                  1151 non-null   object 
 10  (SCA 1, Event)                   1158 non-null   object 
 11  (SCA 2, Player)                  981 non-null    object 
 12  (SCA 2, Event)      

In [18]:
total_df.tail()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                 Minute             Player              Squad   
1255                 87    Virgil van Dijk          Liverpool   
1256                 87       Divock Origi          Liverpool   
1257               90+2         Danny Rose          Tottenham   
1258               90+3      Son Heung-min          Tottenham   
1259               90+3         Harry Kane          Tottenham   

     Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0  \
                     xG               PSxG            Outcome   
1255               0.05                NaN            Blocked   
1256               0.04               0.58               Goal   
1257               0.04               0.20              Saved   
1258               0.04               0.30              Saved   
1259               0.03               0.03              Saved   

     Unnamed: 6_level_0 Unnamed: 7_level_0 Unnamed: 8_level_0       SCA 1  \
               Distance          Body Part              Notes      Player   
1255               15.0         Right Foot             Volley     Alisson   
1256               15.0          Left Foot                NaN  Joël Matip   
1257               25.0         Right Foot                NaN  Danny Rose   
1258               18.0          Left Foot                NaN  Harry Kane   
1259               18.0          Left Foot                NaN  Danny Rose   

                               SCA 2                     Date  
            Event             Player        Event              
1255  Pass (Live)                NaN          NaN  2019-06-01  
1256  Pass (Live)    Virgil van Dijk         Shot  2019-06-01  
1257      Take-On         Danny Rose  Pass (Live)  2019-06-01  
1258  Pass (Live)  Fernando Llorente  Pass (Live)  2019-06-01  
1259  Pass (Live)    Kieran Trippier  Pass (Live)  2019-06-01

In [19]:
# move the last column to index 0
date_column_name = total_df.columns[-1]
date_column = total_df.pop(date_column_name)
total_df.insert(0, date_column_name, date_column)

In [20]:
total_df.columns

MultiIndex([(              'Date',          ''),
            ('Unnamed: 0_level_0',    'Minute'),
            ('Unnamed: 1_level_0',    'Player'),
            ('Unnamed: 2_level_0',     'Squad'),
            ('Unnamed: 3_level_0',        'xG'),
            ('Unnamed: 4_level_0',      'PSxG'),
            ('Unnamed: 5_level_0',   'Outcome'),
            ('Unnamed: 6_level_0',  'Distance'),
            ('Unnamed: 7_level_0', 'Body Part'),
            ('Unnamed: 8_level_0',     'Notes'),
            (             'SCA 1',    'Player'),
            (             'SCA 1',     'Event'),
            (             'SCA 2',    'Player'),
            (             'SCA 2',     'Event')],
           )

In [22]:
df_clean_total = total_df.copy()

In [23]:
df_clean_total.columns = df_clean_total.columns.get_level_values(1)

In [24]:
df_clean_total.columns

Index(['', 'Minute', 'Player', 'Squad', 'xG', 'PSxG', 'Outcome', 'Distance',
       'Body Part', 'Notes', 'Player', 'Event', 'Player', 'Event'],
      dtype='object')

In [25]:
# drop last 4 cols
df_clean_total = df_clean_total.iloc[:, :-4]

In [30]:
df_clean_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260 entries, 0 to 1259
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1260 non-null   object 
 1   Minute     1209 non-null   object 
 2   Player     1209 non-null   object 
 3   Squad      1209 non-null   object 
 4   xG         1208 non-null   float64
 5   PSxG       426 non-null    float64
 6   Outcome    1209 non-null   object 
 7   Distance   1209 non-null   float64
 8   Body Part  1209 non-null   object 
 9   Notes      183 non-null    object 
dtypes: float64(3), object(7)
memory usage: 98.6+ KB


In [29]:
df_clean_total = df_clean_total.rename(columns={"": 'Date'})

In [32]:
df_clean_total.iloc[:, 2].value_counts()

Player
Mohamed Salah             184
Sadio Mané                111
Roberto Firmino            91
Georginio Wijnaldum        42
Trent Alexander-Arnold     41
                         ... 
David Brooks                1
Joshua King                 1
Ben Mee                     1
Jack Cork                   1
Chris Wood                  1
Name: count, Length: 251, dtype: int64

In [34]:
df_clean_total.iloc[:, 1].unique()

array(['17', '18', '19', '24', '25', '27', '30', '36', '37', '45+2', nan,
       '49', '53', '65', '68', '70', '73', '78', '80', '88', '90', '90+1',
       '3', '23', '35', '44', '45', '54', '60', '61', '66', '77', '81',
       '84', '85', '87', '90+3', '5', '6', '9', '15', '31', '34', '46',
       '48', '58', '62', '69', '72', '86', '89', '90+2', 4.0, 10.0, 16.0,
       21.0, 23.0, 26.0, 32.0, 33.0, 39.0, 43.0, 45.0, 50.0, 52.0, 53.0,
       54.0, 56.0, 63.0, 82.0, 86.0, 88.0, '22', '29', '39', '43', '50',
       '63', '67', '75', '7', '10', '26', '28', '32', '40', '82', '83',
       '4', '8', '13', '21', '45+1', '45+3', '11', '47', '59', '64',
       '90+5', 11.0, 38.0, 51.0, 59.0, 60.0, 68.0, 72.0, 75.0, 90.0, 12.0,
       58.0, 61.0, 62.0, 65.0, 69.0, 79.0, '38', '42', '55', '20', '51',
       '56', '76', '90+4', '41', '79', 9.0, 13.0, 14.0, 18.0, 24.0, 25.0,
       31.0, 74.0, 83.0, 84.0, '12', '33', '71', '74', 20.0, 22.0, 28.0,
       29.0, 30.0, 34.0, 35.0, 41.0, 81.0, 85.0, '5

In [204]:
df_clean_total.iloc[:, 0].unique()

array([17., 18., 19., 24., 25., 36., 37., 45., nan, 53., 65., 68., 70.,
       73., 78., 80., 88., 90.,  3., 23., 27., 35., 44., 49., 60., 61.,
       77., 81., 84., 87.,  5.,  9., 15., 31., 34., 46., 58., 69., 72.,
       86.,  4., 10., 16., 21., 43., 54., 82.,  6., 22., 39., 63.,  7.,
       26., 30., 32., 89., 85., 11., 59., 38., 12., 50., 62., 79., 47.,
       64., 20., 48., 51., 56., 76., 13., 41., 42., 66., 74., 83.,  8.,
       33., 71., 28., 67., 29., 57., 52., 14.,  2., 75., 55.,  1.])

In [36]:
def convert_to_int(value):
    if pd.isna(value):
        return value
    if isinstance(value, str):
        return int(value.strip()[:2])
    elif isinstance(value, float):
        return int(value)
    else:
        return None

# Apply the function to the Minutes column
df_clean_total['Minute'] = df_clean_total['Minute'].apply(convert_to_int)

In [37]:
df_clean_total["Minute"] = df_clean_total["Minute"].astype("Int64")

In [38]:
df_clean_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260 entries, 0 to 1259
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1260 non-null   object 
 1   Minute     1209 non-null   Int64  
 2   Player     1209 non-null   object 
 3   Squad      1209 non-null   object 
 4   xG         1208 non-null   float64
 5   PSxG       426 non-null    float64
 6   Outcome    1209 non-null   object 
 7   Distance   1209 non-null   float64
 8   Body Part  1209 non-null   object 
 9   Notes      183 non-null    object 
dtypes: Int64(1), float64(3), object(6)
memory usage: 99.8+ KB


In [39]:
df_clean_total.head()

,Date,Minute,Player,Squad,xG,PSxG,Outcome,Distance,Body Part,Notes
0,2018-08-12,17,Sadio Mané,Liverpool,0.04,NaN,Blocked,16.0,Right Foot,NaN
1,2018-08-12,18,Roberto Firmino,Liverpool,0.59,NaN,Off Target,5.0,Right Foot,Volley
2,2018-08-12,19,Mohamed Salah,Liverpool,0.74,0.95,Goal,4.0,Right Foot,NaN
3,2018-08-12,24,Trent Alexander-Arnold,Liverpool,0.06,0.57,Saved,31.0,Right Foot,Free kick
4,2018-08-12,25,Georginio Wijnaldum,Liverpool,0.09,NaN,Off Target,12.0,Left Foot,NaN


In [40]:
# values for 90 and 45 include shots attempted in extra time (found no better solution)
df_clean_total["Minute"].value_counts()

Minute
90    76
45    29
82    25
85    21
80    20
      ..
19     7
5      4
46     4
40     4
1      3
Name: count, Length: 90, dtype: Int64

In [47]:
df_clean_total.dropna(subset=['Minute'], inplace=True)

In [48]:
df_clean_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1209 entries, 0 to 1259
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1209 non-null   object 
 1   Minute     1209 non-null   Int64  
 2   Player     1209 non-null   object 
 3   Squad      1209 non-null   object 
 4   xG         1208 non-null   float64
 5   PSxG       426 non-null    float64
 6   Outcome    1209 non-null   object 
 7   Distance   1209 non-null   float64
 8   Body Part  1209 non-null   object 
 9   Notes      183 non-null    object 
dtypes: Int64(1), float64(3), object(6)
memory usage: 105.1+ KB


In [49]:
df_clean_total['Player'] = df_clean_total['Player'].str.replace(r'\s*\(pen\)', '', regex=True)

In [53]:
df_clean_total.to_csv("all_shots_CLandPL.csv", index=False)

In [54]:
df_clean_total["Outcome"].unique()

array(['Blocked', 'Off Target', 'Goal', 'Saved', 'Woodwork',
       'Saved off Target'], dtype=object)